# 평가

- 머신러닝 절차: 데이터 가공/변환, 모델 학습/예측, 평가
- 성능 평가 지표는 일반적으로 모델이 분류냐 회귀냐에 따라 여러 종류로 나뉨
- 회귀는 대부분 실제값과 예측값의 오차 평균값에 기반(예: 오차에 절댓값 씌운 뒤 평균 오차, 오차의 제곱 값에 루트를 씌운 뒤 평균 오차)
- 이 장에서는 분류에 사용되는 성능 평가 지표 (특히 0과 1로 결정값이 한정되는 이진 분류의 성능 평가 지표에 대해, 정확도보다 다른 지표가 중요시됨)

<분류 성능 평가 지표>
- 정확도(Accuracy)
- 오차행렬(Confusion Matrix)
- 정밀도(Precision)
- 재현율(Recall)
- F1 스코어
- ROC AUC

- 분류 결정 클래스 값 종류의 유형에 따라 이진 분류와 멀티 분류로 나뉨
- 위 지표 이진/멀티 모두 적용되지만 이진 분류에서 더욱 중요하게 강조

# 01 정확도(Accuracy)

정확도(Accuracy)= 예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수

이진 분류에서 모델 성능 왜곡할 수 있음

타이타닉 예제에서 예측 정확도가 80%였지만 남자인 경우보다 여자인 경우 생존 확률이 높아서 무조건 성별이 여자인 경우 생존, 남자인 경우 사망으로 예측해도 비슷한 수치가 나올 수 있음

In [3]:
from sklearn.base import BaseEstimator

class MyDummyClassfier(BaseEstimator):
    # fit() 메서드는 아무것도 학습하지 않음.
    def fit(self, X, y=None):
        pass
    # predict() 메서드는 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측함.
    def predict(self, X):
        pred = np.zeros((X.shape[0],1))
        for i in range (X.shape[0]):
            if X['Sex'].iloc[i]==1:
                pred[i]=0
            else :
                pred[i]=1
                
        return pred

- pred = np.zeros((X.shape[0],1)), shape(),행의 수, 열의 수, X의 행의 수 만큼
- titanic_df['Sex'].iloc[0] 하면 'male', 'female' 나오는 데 transform_features해서 1과 0이 됨
- class 객체를 이용해서 MyDummyClassifier.fit()처럼 사용할 수 있음

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

이전 주차 transform_features 함수

In [9]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1, inplace=True)
    return df

# 레이블 인코딩 수행.
def format_features(df):
    df['Cabin']=df['Cabin'].str[:1]
    features=['Cabin','Sex','Embarked']
    for feature in features:
        le=LabelEncoder()
        le=le.fit(df[feature])
        df[feature]=le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출

def transform_features(df):
    df=fillna(df)
    df=drop_features(df)
    df=format_features(df)
    return df  

In [10]:
# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할.
titanic_df=pd.read_csv("C:\\Users\\lovej\\Downloads\\kaggle-titanic-master\\kaggle-titanic-master\\input\\titanic_train.csv")
y_titanic_df=titanic_df['Survived']
X_titanic_df=titanic_df.drop('Survived', axis=1)
X_titanic_df=transform_features(X_titanic_df)
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df,
                                                 test_size=0.2, random_state=0)

#  위에서 생성한 Dummy Classfier를 이용해 학습/예측/평가 수행.
myclf=MyDummyClassfier()
myclf.fit(X_train, y_train)

mypredictions=myclf.predict(X_test)
print('Dummy Classfier의 정확도는 : {0:.4f}'.format(accuracy_score(y_test, mypredictions)))

Dummy Classfier의 정확도는 : 0.7877


- 정확도는 이처럼 단순한 알고리즘으로 예측 하더라도 높은 수치가 나올 수 있음
- 특히 불균형한 레이블 값 (90개의 레이블이 0, 10개의 레이블이 1이면 무조건 0으로 예측 결과를 반환하는 모델 정확도가 90% 나옴)

- MNIST 레이블 값이 0~9까지 있는 멀티 레이블 분류, 7인 것만 True, 나머지 값은 False로 변환해 이진 분류 문제로 불균형하게 변환
- 모든 데이터를 False, 0으로 예측하는 classifier를 이용해 정확도 측정하면 90%가까운 예측 정확도
- 무조건 특정한 결과로 찍어도 데이터 분포가 균일하지 않은 경우 높은 수치가 나타날 수 있음

In [21]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    
    #  입력값으로 들어오는 X데이터 세트의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X):
        return np.zeros((len(X),1), dtype=bool)
    
# 사이킷런의 내장 데이터 세트인 load_digits()를 이용해 MNIST 데이터 로딩
digits=load_digits()

# digits 번호가 7번이면 True이고 이를 astype(int)로 1로 변환, 7번이 아니면 False이고 0으로 변환.
y=(digits.target==7).astype(int)
X_train, X_test, y_train, y_test=train_test_split(digits.data,y,random_state=11)

In [24]:
# 불균형한 레이블 데이터 분포도 확인.
print('레이블 테스트 세트 크기:',y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

# Dummy Classifier로 학습/예측/정확도 평가
fakeclf=MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred=fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는:{:.3f}'.format(accuracy_score(y_test, fakepred)))

레이블 테스트 세트 크기: (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64
모든 예측을 0으로 하여도 정확도는:0.900


정확도 평가 지표는 불균형한 레이블 데이터 세트에서 성능 수치로 사용돼서는 안 됨

# 02 오차 행렬

- 이진 분류에서 성능 지표로 잘 활용되는 오차행렬(confusion matrix, 혼동행렬)은 학습된 분류 모델이 예측 수행하면서 얼마나 헷갈리고(confused) 있는지 함께 보여주는 지표

![confusion](https://velog.velcdn.com/images%2Fsset2323%2Fpost%2F2fb704cf-8556-40fc-87a2-75b8feb32986%2Fimage.png)

- TN True Negative : True는 예측 값과 실제 값이 같다는 의미, Negative는 예측값이 Negative 0
- True/False 앞은 실제값이 같은가 틀린가
- Negative/Positive는 예측 결과 값이 0/1인지

In [32]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, fakepred)

array([[405,   0],
       [ 45,   0]], dtype=int64)

- target==7인지 아닌지에 따라 클래스 값을 True/False 이진 분류로 변경
- y_test는 실제값, fakepred는 예측값
- 0이 405건, 1이 45건

정확도=예측 결과와 실제 값이 동일한 건수/전체 데이터 수 =(TN + TP)/(TN + FP + FN + TP)

- 불균형한 레이블 가지는 이진 분류 모델에서는 중점적으로 찾아야 하는 매우 적은 수의 결괏값에 Positive 1부여 (사기 행위, 암이 양성)
- 불균형한 이진 분류 데이터 세트는 Postivie보다 Negative로 예측 정확도 높아짐. TN이 매우 커지고 TP가 매우 작아짐
- Negative로 예측할 때 정확도가 높아 FN이 매우 작아지고, Positivie로 예측하는 경우가 작아 FP도 작아짐
- Positive에 대한 예측 정확도 판단하지 못한 채 Negative에 대하 ㄴ예측 정확도만으로 분류의 정확도가 매우 높게 나타남

# 03 정밀도와 재현율

- 불균형한 데이터에서 정확도보다 더 선호되는 평가 지표
- Positive 데이터의 예측 성능에 초점을 맞춘 평가 지표
- MyFakeClassifier는 Positive로 예측한 TP값이 없어 정밀도와 재현율 모두 0

- 정밀도= TP / (FP + TP)
- 재현율= TP / (FN + TP)

- 정밀도: 예측을 Positive로 한 대상 중에 예측과 실제 값이 Positive로 일치한 데이터의 비율 (양성 예측도)
- 재현율: 실제 값이 Positive인 대상 중에 예측과 실제 값이 Positive로 일치한 데이터의 비율 (민가도, TPR)

- 재현율이 중요한 지표: 실제 Positive 양성 데이터를 Negative로 잘못 판단하면 업무상 큰 영향이 발생하는 경우(암 판단 모델, 금융 사기 적발 모델)
- 정밀도가 중요한 지표: 실제 Negative인 일반 메일을 Positive인 스팸 메일로 분류할 경우 업무 차질(스펨메일 여부)

- 재현율과 정밀도 모두 TP를 높이는 데 동일하게 초점을 맞추지만, 재현율은 FN(실제 Positive, 예측 Negative) 정밀도는 FP를 낮추는 데 초점
- 둘 다 높은 것이 좋지만 어느 한 평가 지표만 매우 높으면 바람직하지 않음

- 정밀도 precision_score()
- 재현율 recall_score()

In [52]:
from sklearn.metrics import accuracy_score, precision_score , recall_score , confusion_matrix

def get_clf_eval(y_test , pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy , precision ,recall))

- liblinear는 로지스틱 회귀의 최적화 알고리즘 유형
- 작은 데이터 세트의 이진 분류인 경우 성능이 약간 좋음
- 기본값이 lbfgs는 상대적으로 크고 다중 분류인 경우 적합

In [53]:
import numpy as np
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression

# 원본 데이터를 재로딩, 데이터 가공, 학습데이터/테스트 데이터 분할. 
titanic_df=pd.read_csv("C:\\Users\\lovej\\Downloads\\kaggle-titanic-master\\kaggle-titanic-master\\input\\titanic_train.csv")
y_titanic_df = titanic_df['Survived']
X_titanic_df= titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, \
                                                    test_size=0.20, random_state=11)

lr_clf = LogisticRegression(solver='liblinear')

lr_clf.fit(X_train , y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test , pred)

오차 행렬
[[108  10]
 [ 14  47]]
정확도: 0.8659, 정밀도: 0.8246, 재현율: 0.7705


## 정밀도/재현율 트레이드 오프

- 정밀도 또는 재현율이 특별이 강조돼야 할 경우 분류의 결정 임곗값(Tjreshold)을 조정해 정밀도 또는 재현율 수치 높일 수 있음
- 하지만 정밀도와 재현율은 상호 보완적인 평가 지표이기 때문에 어느 한 쪽을 강제로 높이면 다른 하나의 수치는 떨어지기 쉬움 (Trade-off)

- 분류 알고리즘은 개별 레이블별로 결정 확률을 구함, 예측 확률이 큰 레이블값으로 예측
- 이진 분류 모델에서 특정 데이터가 0이 될 확률 10%, 1이 될 확률 90%면 90%확률 가진 1로 예측
- 일반적으로 이진 분류에서는 임곗값을 0.5, 즉 50%정하고, 이 기준 값보다 확률이 크면 Positive, 작으면 Negative로 결정

- 사이킷런은 예측 확률을 반환하는 메서드인 predict_proba()제공
- predict_proba()메서드는 학습이 완료된 사이킷런 Classifier객체에서 호출이 가능
- 테스트 피처 데이터 세트를 파라미터로 입력해주면 테스트 피처 레코드의 개별 클래스 예측 확률 반환
- predict()메서드와 유사하지만 단진 반환 결과가 예측 결과 클래스값이 아닌 예측 확률 결과

- 입력 파라미터: predict() 메서드와 동일하게 보통 테스트 피처 데이터 세트를 입력
- 반환 값: 개별 클래스의 예측 확률을 nadrray m x n(m:입력값의 레코드 수, n:클래스 값 유형)형태로 반환
- 입력 테스트 데이터 세트가 표본 개수가 100이고 예측 클래스 값 유형이 2개(이진 분류)라면 반환값은 100x2 nadrray
- 각 열은 개별 클래스의 예측 확률, 이진 분류에서는 첫 번째 칼럼은 0, 두 번째 칼럼은 1 확률

In [58]:
pred_proba=lr_clf.predict_proba(X_test)
pred=lr_clf.predict(X_test)
print('pred_proba()결과 Shape : {0}'.format(pred_proba.shape))
print('pred_prova array에서 앞 3개만 샘플로 추출 \n:', pred_proba[:3])

# 예측 확률 array와 예측 결괏값 array를 병합(concatenate)해 예측 확률과 결괏값을 한눈에 확인
pred_proba_result=np.concatenate([pred_proba, pred.reshape(-1,1)], axis=1)
print('두 개의 class 중에서 더 큰 확률을 클래스 값으로 예측 \n', pred_proba_result[:3])

pred_proba()결과 Shape : (179, 2)
pred_prova array에서 앞 3개만 샘플로 추출 
: [[0.44935225 0.55064775]
 [0.86335511 0.13664489]
 [0.86429643 0.13570357]]
두 개의 class 중에서 더 큰 확률을 클래스 값으로 예측 
 [[0.44935225 0.55064775 1.        ]
 [0.86335511 0.13664489 0.        ]
 [0.86429643 0.13570357 0.        ]]


In [65]:
from sklearn.preprocessing import Binarizer

X = [[1,-1,2],
    [2,0,0],
    [0,1.1,1.2]]

# X의 개별 원소들이 threshold값보다 같거나 작으면 0을, 크면 1을 반환
binarizer=Binarizer(threshold=1.1)
print(binarizer.fit_transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]
